In [1]:
import sys
sys.path.append('..')
sys.path.append('../src')
sys.path.append('../data')
from src.model import WrapHookedTransformer
%load_ext autoreload
%autoreload 2

In [2]:
model = WrapHookedTransformer.from_pretrained("gpt2")

Using pad_token, but it is not set yet.


Loaded pretrained model gpt2 into HookedTransformer


In [3]:
logit, cache = model.run_with_cache("Hello world")

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_43241/2318662754.py:1 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_43241/2318662754.py'                 │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/transformer_lens/Hooked │
│ Transformer.py:552 in run_with_cache                                                      │
│                                                                                           │
│    549 │   │   ActivationCache object, with a bunch of useful HookedTransformer specific  │
│    550 │   │   dictionary of activations as in HookedRootModule.                          │
│    551 │   │   """                                                                        │
│ ❱  552 │   │   out, cache_dict = super().run_with_cache(                                  │
│    553 │   │   │   *model_args, remove_batch_dim=remove_batch_dim, **kwargs               │
│    554 │   │   )                                                                          │
│    555 │   │   if return_cache_object:                                                    │
│                                                                                           │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/transformer_lens/hook_p │
│ oints.py:481 in run_with_cache                                                            │
│                                                                                           │
│   478 │   │   │   reset_hooks_end=reset_hooks_end,                                        │
│   479 │   │   │   clear_contexts=clear_contexts,                                          │
│   480 │   │   ):                                                                          │
│ ❱ 481 │   │   │   model_out = self(*model_args, **model_kwargs)                           │
│   482 │   │   │   if incl_bwd:                                                            │
│   483 │   │   │   │   model_out.backward()                                                │
│   484                                                                                     │
│                                                                                           │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/torch/nn/modules/module │
│ .py:1194 in _call_impl                                                                    │
│                                                                                           │
│   1191 │   │   # this function, and just call forward.                                    │
│   1192 │   │   if not (self._backward_hooks or self._forward_hooks or self._forward_pre_h │
│   1193 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):            │
│ ❱ 1194 │   │   │   return forward_call(*input, **kwargs)                                  │
│   1195 │   │   # Do not call functions when jit is used                                   │
│   1196 │   │   full_backward_hooks, non_full_backward_hooks = [], []                      │
│   1197 │   │   if self._backward_hooks or _global_backward_hooks:                         │
│                                                                                           │
│ /home/francesco/anaconda3/envs/torch/lib/python3.10/site-packages/transformer_lens/Hooked │
│ Transformer.py:480 in forward                                                             │
│                                                                                           │
│    477 │   │   │   │   │   │   devices.get_device_for_block_index(i, self.cfg)            │
│    478 │   │   │   │   │   )                      